In [2]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
import numpy as np
import ast
import joblib
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import hamming_loss, accuracy_score, f1_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
import xgboost as xgb


def preprocess_data():
    print('Preprocessing data...')
    df = pd.read_csv('../data/movies_metadata.csv', low_memory=False)
    df = df[['overview', 'genres']]
    df.dropna(inplace=True)

    # Clean up the genres column
    df['genres'] = df['genres'].apply(ast.literal_eval) \
                              .apply(lambda x: [i['name'] for i in x])

    # Filter out rare genres
    counter = Counter([genre for sublist in df['genres'] for genre in sublist])
    GENRE_FREQ_THRESHOLD = 10
    popular_genres = [genre for genre, count in counter.items() if count >= GENRE_FREQ_THRESHOLD]

    df['genres'] = df['genres'].apply(lambda genres: [genre for genre in genres if genre in popular_genres])

    # Drop rows with no popular genre
    df['genres'] = df['genres'].apply(lambda genres: genres if len(genres) > 0 else np.nan)
    df.dropna(subset=['genres'], inplace=True)

    # One hot encoding of genres
    mlb = MultiLabelBinarizer()
    df_genres = pd.DataFrame(mlb.fit_transform(df.pop('genres')),
                              columns=mlb.classes_,
                              index=df.index)

    joblib.dump(mlb, '../models/mlb.joblib')

    df = df.join(df_genres)
    df.to_csv('../data/processed_movies_metadata.csv', index=False)


def train_model(X_train, y_train, encoder, multilabeler, estimator):
    if encoder == 'TfIdf':
        encoder_model = TfidfVectorizer().fit(X_train)
    elif encoder == 'SentenceTransformer':
        encoder_model = SentenceTransformer('all-MiniLM-L6-v2')
        encoder_model.transform = encoder_model.encode

    X_encoded = encoder_model.transform(X_train)
    classifier = multilabeler(estimator=estimator)
    classifier.fit(X_encoded, y_train)
    return (encoder_model, classifier)

if __name__ == '__main__':
    preprocess_data()
    df = pd.read_csv('../data/processed_movies_metadata.csv')
    X = df['overview']
    y = df.drop(['overview'], axis=1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    # Reset indices so that SentenceTransformer encode doesn't complain
    X_train.reset_index(drop=True, inplace=True)
    y_train.reset_index(drop=True, inplace=True)
    X_test.reset_index(drop=True, inplace=True)
    y_test.reset_index(drop=True, inplace=True)

    print('Training model...')
    encoder = 'SentenceTransformer'
    multilabeler = OneVsRestClassifier
    estimator = LogisticRegression()

    encoder_model, classifier_model = train_model(X_train, y_train,
                                           encoder, multilabeler, estimator)
    y_pred = classifier_model.predict(encoder_model.transform(X_test))
    report = classification_report(y_test, y_pred, output_dict=True)
    print("F1 Score: %.4f"%(report['micro avg']['f1-score']))
    print("Hamming Loss: %.4f"%(hamming_loss(y_test, y_pred)))

    joblib.dump(encoder_model, '../models/encoder_model.joblib')
    joblib.dump(classifier_model, '../models/classifier_model.joblib')

Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> 

KeyboardInterrupt
>>> 
^C


ModuleNotFoundError: No module named 'sentence_transformers'

In [2]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [3]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
import numpy as np
import ast
import joblib
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import hamming_loss, accuracy_score, f1_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
import xgboost as xgb


def preprocess_data():
    print('Preprocessing data...')
    df = pd.read_csv('../data/movies_metadata.csv', low_memory=False)
    df = df[['overview', 'genres']]
    df.dropna(inplace=True)

    # Clean up the genres column
    df['genres'] = df['genres'].apply(ast.literal_eval) \
                              .apply(lambda x: [i['name'] for i in x])

    # Filter out rare genres
    counter = Counter([genre for sublist in df['genres'] for genre in sublist])
    GENRE_FREQ_THRESHOLD = 10
    popular_genres = [genre for genre, count in counter.items() if count >= GENRE_FREQ_THRESHOLD]

    df['genres'] = df['genres'].apply(lambda genres: [genre for genre in genres if genre in popular_genres])

    # Drop rows with no popular genre
    df['genres'] = df['genres'].apply(lambda genres: genres if len(genres) > 0 else np.nan)
    df.dropna(subset=['genres'], inplace=True)

    # One hot encoding of genres
    mlb = MultiLabelBinarizer()
    df_genres = pd.DataFrame(mlb.fit_transform(df.pop('genres')),
                              columns=mlb.classes_,
                              index=df.index)

    joblib.dump(mlb, '../models/mlb.joblib')

    df = df.join(df_genres)
    df.to_csv('../data/processed_movies_metadata.csv', index=False)


def train_model(X_train, y_train, encoder, multilabeler, estimator):
    if encoder == 'TfIdf':
        encoder_model = TfidfVectorizer().fit(X_train)
    elif encoder == 'SentenceTransformer':
        encoder_model = SentenceTransformer('all-MiniLM-L6-v2')
        encoder_model.transform = encoder_model.encode

    X_encoded = encoder_model.transform(X_train)
    classifier = multilabeler(estimator=estimator)
    classifier.fit(X_encoded, y_train)
    return (encoder_model, classifier)

if __name__ == '__main__':
    preprocess_data()
    df = pd.read_csv('../data/processed_movies_metadata.csv')
    X = df['overview']
    y = df.drop(['overview'], axis=1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    # Reset indices so that SentenceTransformer encode doesn't complain
    X_train.reset_index(drop=True, inplace=True)
    y_train.reset_index(drop=True, inplace=True)
    X_test.reset_index(drop=True, inplace=True)
    y_test.reset_index(drop=True, inplace=True)

    print('Training model...')
    encoder = 'SentenceTransformer'
    multilabeler = OneVsRestClassifier
    estimator = LogisticRegression()

    encoder_model, classifier_model = train_model(X_train, y_train,
                                           encoder, multilabeler, estimator)
    y_pred = classifier_model.predict(encoder_model.transform(X_test))
    report = classification_report(y_test, y_pred, output_dict=True)
    print("F1 Score: %.4f"%(report['micro avg']['f1-score']))
    print("Hamming Loss: %.4f"%(hamming_loss(y_test, y_pred)))

    joblib.dump(encoder_model, '../models/encoder_model.joblib')
    joblib.dump(classifier_model, '../models/classifier_model.joblib')

Preprocessing data...


FileNotFoundError: [Errno 2] No such file or directory: '../data/movies_metadata.csv'